### import of packages

In [1]:
import numpy as np # better arrays than inbuilt arrays
import matplotlib.pyplot as plt # to plot stuff

import pandas as pd #for DataFrame tables
from IPython.display import display #to display dfs more nicely

import scipy.stats
#from scipy.stats import norm
import statistics

## Functions

### Generation of random distributions from 3 Gaussians and plotting them

In [2]:
def create_data_from_3_gaussians(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3): # Creation of random data points from multiple Gaussians ki
    k1 = np.random.normal(mean1, std1, n1) #creates array with values created through Gaussian
    k2 = np.random.normal(mean2, std2, n2)
    k3 = np.random.normal(mean3, std3, n3)

    return np.concatenate([k1, k2, k3])

In [3]:
def p_xi_given_zj_from_gaussian_datapoints(kall, mean1, mean2, mean3, std1, std2, std3):
    # Calculating p_xi_given_zj (this is what Matt is working on with the simulated data)

    pdf_probability_k1 = scipy.stats.norm.pdf(kall, loc=mean1, scale=std1)
    pdf_probability_k2 = scipy.stats.norm.pdf(kall, loc=mean2, scale=std2)
    pdf_probability_k3 = scipy.stats.norm.pdf(kall, loc=mean3, scale=std3)

    p_xi_given_zj = np.vstack((pdf_probability_k1,pdf_probability_k2,pdf_probability_k3))
    p_xi_given_zj = np.transpose(p_xi_given_zj)

    print("p_xi_given_zj")
    display(pd.DataFrame(p_xi_given_zj))
    print("\n")
    
    print("likeliest peptide z of each datapoint x according to scipy.stats.norm.pdf (should be more accurate than EM because its dedicated to Gaussians)")
    print(np.argmax(p_xi_given_zj, axis=1)) # for each column of P(Xi|Zj), the most likely Peptide is returned
    print("\n")
    
    return p_xi_given_zj

In [4]:
def create_3_gaussians_and_calculate_p_xi_given_zj(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3):
    kall = create_data_from_3_gaussians(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3)
    p_xi_given_zj = p_xi_given_zj_from_gaussian_datapoints(kall, mean1, mean2, mean3, std1, std2, std3)
    
    return p_xi_given_zj

In [5]:
def plot_histograms_and_pdfs_from_gaussians(mean1, mean2, mean3, std1, std2, std3, n1, n2, n3):
    k1 = np.random.normal(mean1, std1, n1) #creates array with values created through Gaussian
    k2 = np.random.normal(mean2, std2, n2)
    k3 = np.random.normal(mean3, std3, n3)
    
    # plotting histograms
    nbins = 50
    plt.hist(k1, label = "Peptide 0", bins=nbins, alpha=0.3, density=True, color="orange") # alpha=transparency, density=True normalises to 1 
    plt.hist(k2, label = "Peptide 1", bins=nbins, alpha=0.3, density=True, color="green")
    plt.hist(k3, label = "Peptide 2", bins=nbins, alpha=0.3, density=True, color="blue")

    # PDF plot
    xmin, xmax = plt.xlim() #finds lower and upper bounds of histogram data
    x = np.linspace(start=xmin, stop=xmax, num=100) #num is the number of returned data points - the more points, the finer the fit is plotted
    p1 = scipy.stats.norm.pdf(x, mean1, std1)
    p2 = scipy.stats.norm.pdf(x, mean2, std2)
    p3 = scipy.stats.norm.pdf(x, mean3, std3)

    plt.plot(x, p1, linewidth=2, color = "orange", label = "Gauss function k1: mean = {:.2f}, STD = {:.2f}".format(mean1, std1))
    plt.plot(x, p2, linewidth=2, color = "green", label = "Gauss function k2: mean = {:.2f}, STD = {:.2f}".format(mean2, std2))
    plt.plot(x, p3, linewidth=2, color = "blue", label = "Gauss function k3: mean = {:.2f}, STD = {:.2f}".format(mean3, std3))

    plt.legend(loc='upper right')
    plt.title("PDFs of dwarves and humans")

    plt.show()

### EM    

First iteration:
- assume P(Zi)=1/m for all i

Real data: a bunch of dye seq data points Xi.

Goal: Figure out probability of real peptide seqs Zj given all those measured Xi.
____
However, I am starting with the list of P(Xi|Zj) that Matt will provide me, instead of a list of P(Xi). In fact, I never even use P(Xi), which is a little weird.

In [6]:
def update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi):
    denominator = 0

    for i, row in enumerate(p_xi_given_zj): # Calculating/Updating P(Zj|Xi)
        #print("ROW of P(Xi|Zj):", i)
        for j, cell in enumerate(row):
            #print("COLUMN of P(Xi|Zj):", j)
            numerator = cell * p_zj[j]
            #print("numerator:", numerator)
            
            for l, cell in enumerate(p_zj):
                # print("cell", i, l, p_xi_given_zj[i, l], end="")
                # print(" * zl", p_zj[l])
                denominator = denominator + p_xi_given_zj[i, l] * p_zj[l]
            p_zj_given_xi[j][i] = numerator/denominator
            denominator = 0                                
        #display(pd.DataFrame(p_zj_given_xi))
        
    #print((np.argmax(p_zj_given_xi, axis=1))) # reports index of max value from each row
    
    return p_zj_given_xi

In [7]:
def update_p_zj(p_zj, p_zj_given_xi):    
    for j, element in enumerate(p_zj): #updating the expectation value of Zi
        p_zj[j] = p_zj_given_xi[j].sum()/p_xi_given_zj.shape[1] # divide by number of peptides z
        
    # print("p_zj")
    # display(pd.DataFrame(p_zj))
    
    return p_zj

In [8]:
def EM(p_xi_given_zj):
    ### initialise p_zj_given_xi
    # p_xi_given_zj = np.arange(0,102,1).reshape(34,3) # for testing EM iterating through columns and rows
    p_zj_given_xi = np.full((p_xi_given_zj.shape[1], p_xi_given_zj.shape[0]), 0, dtype=float) #Initialisation based on array size of p_xi_given_zj -- same size, but transposed
    #display(pd.DataFrame(p_zj_given_xi))
    
    ### initialise p_zj
    n = p_xi_given_zj.shape[1] # number of peptides
    p_zj_initial = 1/n #initial approximation: all zj equally likely, to jumpstart first iteration
    p_zj = np.full(n, p_zj_initial)
    
    loopcounter = 0
    while loopcounter <= 100:
        p_zj_given_xi = update_p_zj_given_xi(p_xi_given_zj, p_zj, p_zj_given_xi)
        p_zj = update_p_zj(p_zj, p_zj_given_xi)
        
        loopcounter = loopcounter + 1
        
    print("P_zj_given_xi:")
    display(pd.DataFrame(p_zj_given_xi))
    print("\n")
    
    print("P_zj:")
    display(pd.DataFrame(p_zj))
    p_zj_fraction = p_zj/np.sum(p_zj)
    print("P_zj_fraction")
    display(pd.DataFrame(p_zj_fraction))
    print("\n")
    
    print("Likeliest peptide z for each datapoint x:")
    print((np.argmax(p_zj_given_xi, axis=0))) # reports index of max value from each row
    print("\n")
    
    print("Value of likeliest z:")
    print(np.amax(p_zj_given_xi, axis=0)) #reports max value from each row
    print("\n")
    
    return p_zj

## bootstrapping

Takes in full p_xi_given_zj dataset, and then returns subarray of new p_xi_given_zj

In [105]:
def create_subarray_of_p_xi_given_zj(p_xi_given_zj, bootstrap_sampled_fraction):
    df_p_xi_given_zj = pd.DataFrame(p_xi_given_zj)
    df_p_xi_given_zj_sample = df_p_xi_given_zj.sample(frac=bootstrap_sampled_fraction, axis='rows', replace=True) # filters for a random partial dataset
    display(pd.DataFrame(df_p_xi_given_zj_sample))
    p_xi_given_zj_subarray = df_p_xi_given_zj_sample.to_numpy()
    
    display(pd.DataFrame(p_xi_given_zj_subarray))
    
    return p_xi_given_zj_subarray

In [106]:
def bootstrap_EM(n_bootstrap_runs, bootstrap_sampled_fraction):
    i = 0
    
    p_zj_bootstrap = np.full((p_xi_given_zj.shape[1]), 0, dtype=float)
    print("p_zj_bootstrap", p_zj_bootstrap)
    
    while i < n_bootstrap_runs:
        p_xi_given_zj_subarray = create_subarray_of_p_xi_given_zj(p_xi_given_zj, bootstrap_sampled_fraction)
        
        if i == 0:
            p_zj_bootstrap = EM(p_xi_given_zj_subarray)
            
        else:
            p_zj_bootstrap = np.vstack((p_zj_bootstrap, EM(p_xi_given_zj_subarray)))
        
        i = i + 1
    return p_zj_bootstrap

## call functions

In [110]:
# esc, ctrl+a, ctrl+enter to run all cells

#p_xi_given_zj = create_3_gaussians_and_calculate_p_xi_given_zj(140, 150, 160, 15, 15, 15, 5, 5, 5) #mean1-3, std1-3, n1-3
# plot_histograms_and_pdfs_from_gaussians(140, 150, 160, 15, 15, 15, 5, 5, 5)

p_xi_given_zj = np.genfromtxt('uniform-2-peps-scores.csv', delimiter=',') # import full dataset
p_zj_bootstrap_results = bootstrap_EM(30, 0.7) # n bootstrap runs, fraction of sampled data per run

p_zj_bootstrap [0. 0.]


,0,1
6,609.576654,609.576628
0,442.788477,3504.082019
9,40924.985131,4458.562523
1,353113.978894,38469.916140
5,79290.092111,8638.239712
4,13198.296557,1437.885200
7,83233.660212,9067.870800


,0,1
0,609.576654,609.576628
1,442.788477,3504.082019
2,40924.985131,4458.562523
3,353113.978894,38469.916140
4,79290.092111,8638.239712
5,13198.296557,1437.885200
6,83233.660212,9067.870800


P_zj_given_xi:


,0,1,2,3,4,5,6
0,0.93349,0.639452,0.992298,0.992298,0.992298,0.992298,0.992298
1,0.06651,0.360548,0.007702,0.007702,0.007702,0.007702,0.007702


...



P_zj:


,0
0,3.500000e+00
1,1.276609e-63


P_zj_fraction


,0
0,1.000000e+00
1,3.647456e-64




Likeliest peptide z for each datapoint x:
[0 0 0 0 0 0 0]


Value of likeliest z:
[1. 1. 1. 1. 1. 1. 1.]




## output

In [111]:
p_zj_bootstrap_results_fraction = p_zj_bootstrap_results/np.sum(p_zj_bootstrap_results, axis = 1)[0]
print("All zj bootstrapping values: ")
display(pd.DataFrame(p_zj_bootstrap_results_fraction))

p_zj_bootstrap_results_fraction_avg = np.sum(p_zj_bootstrap_results_fraction, axis = 0)/p_zj_bootstrap_results.shape[0]
print("AVG zj bootstrapping values: \n", p_zj_bootstrap_results_fraction_avg, sep="")

p_zj_bootstrap_results__fraction_std = np.std(p_zj_bootstrap_results_fraction, axis = 0)
print("STD of average zj bootstrapping values: \n", p_zj_bootstrap_results__fraction_std, sep="")

for i, avg_value in enumerate(p_zj_bootstrap_results_fraction_avg):
    print("Peptide", i, ": ", avg_value, "±", p_zj_bootstrap_results__fraction_std[i])

All zj bootstrapping values: 


,0,1
0,0.933490,6.651000e-02
1,1.000000,3.647456e-64
2,0.933490,6.651000e-02
3,1.000000,3.647465e-64
4,1.000000,3.601833e-32
5,0.963654,3.634558e-02
6,1.000000,3.647465e-64
7,1.000000,3.647456e-64
8,0.384497,6.155025e-01
9,0.933490,6.651000e-02


AVG zj bootstrapping values: 
[0.82877326 0.17122674]
STD of average zj bootstrapping values: 
[0.1926491 0.1926491]
Peptide 0 :  0.8287732621419402 ± 0.19264910183107203
Peptide 1 :  0.17122673785805959 ± 0.192649101831072
